## Parse Input

In [7]:
import re
from collections import namedtuple

Fold = namedtuple('Fold', ['direction', 'position'])
Coord = namedtuple('Coord', ['x', 'y'])

REGEX_COORD = r'^(\d+),(\d+)$'
REGEX_FOLD = r'^fold along ([xy])=(\d+)'

def parse_input(input_lines):
  coords = [
    Coord(
      int(re.match(REGEX_COORD, coord).group(1)),
      int(re.match(REGEX_COORD, coord).group(2))
    )
    for coord in
    filter(lambda l: re.match(REGEX_COORD, l) is not None, input_lines)
  ]

  folds = [
    Fold(
      re.match(REGEX_FOLD, fold).group(1),
      int(re.match(REGEX_FOLD, fold).group(2))
    )
    for fold in
    filter(lambda l: re.match(REGEX_FOLD, l) is not None, input_lines)
  ]
  
  return coords, folds


## Build Grid

In [8]:
from typing import List, Tuple

def set_cell(grid: List[List[bool]], x: int, y: int, value: bool):
  grid[y][x] = value

def create_grid(coord_list: List[Coord]) -> List[List[bool]]:
  x_len = max(map(lambda c: c.x, coord_list)) + 1
  y_len = max(map(lambda c: c.y, coord_list)) + 1
  grid = [[False for x in range(x_len)] for y in range(y_len)]
  for coord in coord_list:
    set_cell(grid, coord.x, coord.y, True)
  return grid

def render_grid(grid: List[List[bool]]):
  str_repr = ''
  for row in grid:
    for cell in row:
      str_repr += '# ' if cell else '. '
    str_repr += '\n'
  return str_repr



In [9]:
from collections import Counter

def or_lists(l1: List[bool], l2: List[bool]) -> List[bool]:
  return list(map(lambda l: l[0] or l[1], zip(l1, l2)))

def fold_y(grid: List[List[bool]], y_pos: int) -> List[List[bool]]:
  new_grid = []
  i = 1
  while i <= y_pos:
    new_grid.insert(0, or_lists(grid[y_pos - i], grid[y_pos + i]))
    i += 1
  return new_grid

def fold_line(line: List[bool], x_pos: int) -> List[bool]:
  l1 = line[:x_pos]
  l2 = reversed(line[x_pos+1:])
  return or_lists(l1, l2)

def fold_x(grid: List[List[bool]], x_pos: int) -> List[List[bool]]:
  new_grid = []
  for row in grid:
    new_grid.append(fold_line(row, x_pos))
  return new_grid

def fold(grid: List[List[bool]], fold: Fold) -> List[List[bool]]:
  if fold.direction == 'x':
    return fold_x(grid, fold.position)
  elif fold.direction == 'y':
    return fold_y(grid, fold.position)

In [10]:
with open('./input13') as input:
  input_lines = [line.strip() for line in input.readlines()]

coords, folds = parse_input(input_lines)

grid = create_grid(coords)

folded = fold(grid, folds[0])

dot_count = Counter(sum(folded, []))[True]

print(f'Solution 1 is {dot_count}')

Solution 1 is 664


In [11]:
folded = grid
for f in folds:
  folded = fold(folded, f)

print(render_grid(folded))

# # # # . # # # # . . . # # . # . . # . # # # # . # . . . . # # # . . # . . . . 
# . . . . # . . . . . . . # . # . # . . . . . # . # . . . . # . . # . # . . . . 
# # # . . # # # . . . . . # . # # . . . . . # . . # . . . . # # # . . # . . . . 
# . . . . # . . . . . . . # . # . # . . . # . . . # . . . . # . . # . # . . . . 
# . . . . # . . . . # . . # . # . # . . # . . . . # . . . . # . . # . # . . . . 
# # # # . # . . . . . # # . . # . . # . # # # # . # # # # . # # # . . # # # # . 

